# COVID-19 Dashboard
This dashboard visualises COVID-19 statistics, including daily cases and deaths. 

## Instructions
- Use the dropdown menu to select the metric to display.
- Click "Refresh Data" to fetch the latest statistics from the UKHSA API.

In [ ]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
import json

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

with open("data.json", "r") as f:
    local_data = json.load(f)

def wrangle_data(rawdata):
    df = pd.DataFrame(rawdata["data"])
    df["date"] = pd.to_datetime(df["date"], format='%d/%m/%Y')
    df.set_index('date', inplace=True)
    for col in ['newCases', 'cumCases', 'newDeaths', 'cumDeaths']:
        if col not in df.columns:
            df[col] = np.nan 
    return df

df = wrangle_data(local_data)

# fetch data from the UKHSA API
def fetch_api_data():
    base_url = "https://api.ukhsa-dashboard.data.gov.uk"
    structure = {
        "theme": "infectious_disease",
        "sub_theme": "respiratory",
        "topic": "COVID-19",
        "geography_type": "Nation",
        "geography": "England",
        "metric": "COVID-19_cases_casesByDay"
    }
    url = f"{base_url}/themes/{structure['theme']}/sub_themes/{structure['sub_theme']}/topics/{structure['topic']}/geography_types/{structure['geography_type']}/geographies/{structure['geography']}/metrics/{structure['metric']}"
    try:
        time.sleep(0.34)
        response = requests.get(url)
        response.raise_for_status()
        api_data = response.json()
        print("Data successfully fetched from the API!")
        return api_data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from API: {e}")
        return None


def process_api_data(api_data):
    api_df = pd.DataFrame(api_data["results"])
    api_df["date"] = pd.to_datetime(api_df["date"], format='%Y-%m-%d')
    api_df.set_index('date', inplace=True)
    for col in ['newCases', 'cumCases', 'newDeaths', 'cumDeaths']:
        if col not in api_df.columns:
            api_df[col] = np.nan
    return api_df

def update_data():
    global df
    api_data = fetch_api_data()
    if api_data:
        api_df = process_api_data(api_data)
        df = pd.concat([df, api_df]).drop_duplicates().sort_index()
        print("Data has been updated with API data.")
    else:
        print("Using local data due to API fetch error.")

# create a button to refresh the data
def refresh_button_callback(button):
    update_data()
    refresh_graph()
    refresh_button.description = 'Data Refreshed'
    refresh_button.disabled = True

refresh_button = wdg.Button(
    description='Refresh Data',
    disabled=False,
    button_style='primary',
    tooltip='Click to fetch current data from UKHSA API',
    icon='refresh'
)
refresh_button.on_click(refresh_button_callback)

display(refresh_button)

metric_selector = wdg.Dropdown(
    options={
        "New Cases": "newCases",
        "Cumulative Cases": "cumCases",
        "New Deaths": "newDeaths",
        "Cumulative Deaths": "cumDeaths"
    },
    value="newCases",
    description="Select Metric:",
    style={'description_width': 'initial'}
)

output = wdg.Output()

def plot_data(metric):
    with output:
        clear_output(wait=True)
        plt.figure(figsize=(10,6))
        plt.plot(df.index, df[metric], marker='o', linestyle='-')
        plt.title(f"{metric_selector.label} over Time")
        plt.xlabel("Date")
        plt.ylabel("Number")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

def refresh_graph():
    plot_data(metric_selector.value)

metric_selector.observe(lambda change: refresh_graph(), names='value')

display(metric_selector, output)

refresh_graph()

Button(button_style='primary', description='Refresh Data', icon='refresh', style=ButtonStyle(), tooltip='Click…

Dropdown(description='Select Metric:', options={'New Cases': 'newCases', 'Cumulative Cases': 'cumCases', 'New …

Output()